In [4]:
import datasets
from datasets import load_dataset
import json

In [24]:
data_=[]
with open('/data/MaoXiaowei/models/sier/data/extend_data_finish_46k.jsonl') as f:
    for line in f:
        data_.append(json.loads(line))

In [35]:
data_[1]

{'deepseek_reasoning': '1. Hypothesize \\( x * y = 1 + xy \\).\n2. Verify \\( x * 0 = 1 \\) holds.\n3. Verify \\( (x * y) * z = (z * (xy)) + z \\) holds.\n4. Compute \\( 31 * 32 = 1 + 31 \\cdot 32 = 993 \\).',
 'deepseek_solution': '\\boxed{993}',
 'prompt': [{'content': 'Please reason step by step and put the final answer in \\boxed{}.',
   'role': 'system'},
  {'content': 'In the set of real numbers $\\mathbf{R}$, define the operation $*$ satisfying:\n(i) $x * 0 = 1$ (for any $x \\in \\mathbf{R}$).\n(ii) $(x * y) * z = (z * x y) + z$ (for any $x, y, z \\in $\\mathbf{R}$).\nFind $31 * 32$.',
   'role': 'user'}],
 'domain': 'math',
 'language': 'en',
 'data_id': 0,
 'prompt_id': 1}

In [31]:
from transformers import AutoTokenizer

In [32]:
path='/data/MaoXiaowei/models/model/deepseek-ai/DeepSeek-R1-Distill-Qwen-1___5B/'
tokenizer = AutoTokenizer.from_pretrained(path, trust_remote_code=True)

In [37]:
dataset = datasets.Dataset.from_list(data_)

In [38]:
dataset

Dataset({
    features: ['deepseek_reasoning', 'deepseek_solution', 'prompt', 'domain', 'language', 'data_id', 'prompt_id', 'length'],
    num_rows: 46650
})

In [39]:
# 转换为数据集
def map_function(example):
    output_ids = tokenizer(example['deepseek_reasoning']+example['deepseek_solution']).input_ids
    example['length'] = len(output_ids)
    return example

# 使用数据集的map方法
dataset = dataset.map(map_function)

Map:   0%|          | 0/46650 [00:00<?, ? examples/s]

In [43]:
# 根据 length 对数据集进行排序
dataset = dataset.sort('length')

KeyboardInterrupt: 

In [50]:
# 添加 rank 列
dataset = dataset.map(lambda x, i: {'rank': i/len(dataset)}, with_indices=True)


Map:   0%|          | 0/46650 [00:00<?, ? examples/s]

In [67]:
dataset[-1000]

{'deepseek_reasoning': 'Okay, I need to solve this problem where I have to find the number of unique divisors of a friendly number that are not divisors of any of the unfriendly numbers. Let\'s think through how to approach this.\n\nFirst, the problem says that the friendly number is f, and there are n unfriendly numbers. The task is to find all divisors of f that do not divide any of the unfriendly numbers. Then, count how many such divisors there are.\n\nHmm. Let\'s start by understanding the sample input. In the sample, the friendly number is 16. Its divisors are 1, 2, 4, 8, 16. Then, for each of these, check if any of the unfriendly numbers (like 2,5,7, etc.) are divisible by these divisors. Wait, no—wait, wait. Wait, no. Wait, the divisors of the friendly number are possible candidates. For each such divisor d (of f), we need to check whether d divides any of the unfriendly numbers. Because if d divides any unfriendly number, then d is a common divisor and thus should be excluded.

In [70]:
dataset[100]

{'deepseek_reasoning': '总时间30分钟，减去到学校大门的15分钟，再减去到学校楼的6分钟，剩余时间为9分钟。',
 'deepseek_solution': '\\boxed{9}',
 'prompt': [{'content': '请一步步推理，并把最终答案放到 \\boxed{}。', 'role': 'system'},
  {'content': '我和我弟弟有30分钟去学校，不能迟到。我们到学校大门需要15分钟，再花6分钟到学校楼。我们还有多少时间到我们的教室，不会迟到？',
   'role': 'user'}],
 'domain': 'math',
 'language': 'zh',
 'data_id': 5822,
 'prompt_id': 4,
 'length': 36,
 'rank': 0.0021436227224008574}

In [77]:
new_dataset=dataset.select(range(100,len(dataset)-1000))

In [78]:
new_dataset

Dataset({
    features: ['deepseek_reasoning', 'deepseek_solution', 'prompt', 'domain', 'language', 'data_id', 'prompt_id', 'length', 'rank'],
    num_rows: 45550
})

In [79]:
new_dataset=new_dataset.map(lambda x, i: {'rank': i/len(new_dataset)}, with_indices=True)

Map:   0%|          | 0/45550 [00:00<?, ? examples/s]

In [86]:
# 保存数据集为 parquet 格式
new_dataset.save_to_disk('/data/MaoXiaowei/models/adre/dataset/mix_filtered_45k')


Saving the dataset (0/1 shards):   0%|          | 0/45550 [00:00<?, ? examples/s]